In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from datetime import datetime, timedelta, timezone

In [2]:
failure_2016 = pd.read_csv('./data/init/failures-2016.csv',sep=';')
failure_2017 = pd.read_csv('./data/init/failures-2017.csv',sep=';')
metmast_2016 = pd.read_csv('./data/init/metmast-2016.csv',sep=';')
metmast_2017 = pd.read_csv('./data/init/metmast-2017.csv',sep=';')
signals_2016 = pd.read_csv('./data/init/signals-2016.csv', sep=';')
signals_2017= pd.read_csv('./data/init/signals-2017.csv', sep=';')



# 1. Cleaning Signal data

Combining signals from both years and aggregating the time series data into recurring once a day.

In [3]:
def signal_preprocess(signals):
    
    signals['Timestamp'] = pd.to_datetime(signals['Timestamp'])
    signals=signals.set_index('Timestamp')
    return signals


In [4]:
signals=pd.concat([signals_2016, signals_2017], axis=0)
signals = signal_preprocess(signals)

Drop the columns with low variance

In [5]:
def get_signals_with_low_variance(df: pd.DataFrame, threshold=0) -> list:
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    cont_data = df.select_dtypes(include=numerics)
    selector = VarianceThreshold(threshold=threshold)
    selector.fit(cont_data)
    inverted_list = ~np.array(selector.get_support())
    return cont_data.columns[inverted_list].tolist()

In [6]:
cols_to_drop=get_signals_with_low_variance(signals)
signals.drop(cols_to_drop, axis=1, inplace=True)

Aggregate the time series dataframe into a daily data frame

In [7]:
def aggregate_signals(signals):
    agg_signals=signals.groupby('Turbine_ID').resample('D').sum()
    agg_signals['Turbine_ID'] = agg_signals.index.get_level_values('Turbine_ID')  
    agg_signals=agg_signals.reset_index('Timestamp')
    return agg_signals


In [8]:
agg_signals=aggregate_signals(signals)
agg_signals=agg_signals.reset_index(drop=True)

In [9]:
agg_signals

,Timestamp,Turbine_ID,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,...,Grd_Prod_PsbleInd_Avg,Grd_Prod_PsbleInd_Max,Grd_Prod_PsbleInd_Min,Grd_Prod_PsbleInd_Std,Grd_Prod_PsbleCap_Avg,Grd_Prod_PsbleCap_Max,Grd_Prod_PsbleCap_Min,Grd_Prod_PsbleCap_Std,Gen_Bear2_Temp_Avg,Nac_Direction_Avg
0,2016-01-01 00:00:00+00:00,T01,181289.2,133914.9,158499.3,13332.7,5667.0,8128,8126,8049,...,-92929.7,-68241.0,-109983.9,9823.7,92920.8,109983.9,67537.9,9877.8,5554,32453.4
1,2016-01-02 00:00:00+00:00,T01,100793.6,71082.2,86946.4,8378.4,4907.0,6556,6525,6458,...,-45063.8,-35598.1,-53448.7,4432.3,44758.1,53448.7,33839.1,4969.9,4752,23121.9
2,2016-01-03 00:00:00+00:00,T01,133818.3,108905.3,124205.1,5775.7,6162.0,7969,7768,7627,...,-67850.8,-54531.9,-73000.0,5227.6,63328.1,73000.0,42621.4,8344.0,5503,35495.5
3,2016-01-04 00:00:00+00:00,T01,246412.7,215913.9,234245.0,5730.7,9402.0,14207,13658,13447,...,-133426.6,-100994.2,-143994.0,13072.6,119419.5,143390.4,77022.7,19260.8,8811,38177.5
4,2016-01-05 00:00:00+00:00,T01,240840.6,206798.4,225681.7,7157.1,8107.0,13162,12695,12515,...,-129142.2,-116437.2,-142871.2,6964.7,119370.2,140043.4,101280.0,8765.6,7097,45004.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2919,2017-12-27 00:00:00+00:00,T11,243986.1,205969.3,228579.7,8038.6,9281.0,12396,12417,12362,...,-120201.6,-97406.6,-143258.6,13087.6,107426.6,140996.6,76593.5,15766.5,8355,40865.5
2920,2017-12-28 00:00:00+00:00,T11,237997.3,192192.0,220250.6,10931.2,8367.0,10300,10401,10322,...,-135333.5,-109675.3,-143738.0,8928.4,130039.6,143738.0,94313.4,12493.0,6940,39614.2
2921,2017-12-29 00:00:00+00:00,T11,188502.9,146124.7,165452.9,13330.9,5920.0,7934,8103,8036,...,-85117.9,-37053.6,-122976.5,20889.7,85117.9,122976.5,37053.6,20889.7,5539,43819.8
2922,2017-12-30 00:00:00+00:00,T11,65884.7,51194.5,57913.6,4789.3,4327.0,5416,5475,5453,...,-12643.7,-1886.9,-30372.6,6473.7,12643.7,30372.6,1886.9,6473.7,4220,22927.5


Prepare signal for each turbine

In [50]:
"""turbine_names = agg_signals["Turbine_ID"].unique()
def create_df_for_each_turbine(signals: pd.DataFrame) -> list[pd.DataFrame]:
    turbine_dfs = list()

    for turbine in turbine_names:
        test = agg_signals["Turbine_ID"]
        turbine_df = signals[agg_signals["Turbine_ID"] == turbine]
        turbine_df = turbine_df.sort_values("Timestamp")
        turbine_df = turbine_df.reset_index()
        
        turbine_dfs.append(turbine_df)

    return turbine_dfs

turbine_dfs = create_df_for_each_turbine(agg_signals)
#convert list to dataframe
#turbine_signals = pd.concat(turbine_dfs, axis=0)"""


In [12]:
#create a binary column to indicate if a signal data has any missing values
agg_signals['missing_values'] = agg_signals.isnull().any(axis=1).astype(int)
#fill the missing valeu with backfill and forwardfill
signals_clean=agg_signals.fillna(method='bfill')
#signals_clean=signals_clean.reset_index()

/var/folders/7b/1zsp53s91z97clkc70s5xcym0000gn/T/ipykernel_31723/173728987.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  signals_clean=agg_signals.fillna(method='bfill')


In [13]:
signals_clean

,Timestamp,Turbine_ID,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,...,Grd_Prod_PsbleInd_Max,Grd_Prod_PsbleInd_Min,Grd_Prod_PsbleInd_Std,Grd_Prod_PsbleCap_Avg,Grd_Prod_PsbleCap_Max,Grd_Prod_PsbleCap_Min,Grd_Prod_PsbleCap_Std,Gen_Bear2_Temp_Avg,Nac_Direction_Avg,missing_values
0,2016-01-01 00:00:00+00:00,T01,181289.2,133914.9,158499.3,13332.7,5667.0,8128,8126,8049,...,-68241.0,-109983.9,9823.7,92920.8,109983.9,67537.9,9877.8,5554,32453.4,0
1,2016-01-02 00:00:00+00:00,T01,100793.6,71082.2,86946.4,8378.4,4907.0,6556,6525,6458,...,-35598.1,-53448.7,4432.3,44758.1,53448.7,33839.1,4969.9,4752,23121.9,0
2,2016-01-03 00:00:00+00:00,T01,133818.3,108905.3,124205.1,5775.7,6162.0,7969,7768,7627,...,-54531.9,-73000.0,5227.6,63328.1,73000.0,42621.4,8344.0,5503,35495.5,0
3,2016-01-04 00:00:00+00:00,T01,246412.7,215913.9,234245.0,5730.7,9402.0,14207,13658,13447,...,-100994.2,-143994.0,13072.6,119419.5,143390.4,77022.7,19260.8,8811,38177.5,0
4,2016-01-05 00:00:00+00:00,T01,240840.6,206798.4,225681.7,7157.1,8107.0,13162,12695,12515,...,-116437.2,-142871.2,6964.7,119370.2,140043.4,101280.0,8765.6,7097,45004.3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2919,2017-12-27 00:00:00+00:00,T11,243986.1,205969.3,228579.7,8038.6,9281.0,12396,12417,12362,...,-97406.6,-143258.6,13087.6,107426.6,140996.6,76593.5,15766.5,8355,40865.5,0
2920,2017-12-28 00:00:00+00:00,T11,237997.3,192192.0,220250.6,10931.2,8367.0,10300,10401,10322,...,-109675.3,-143738.0,8928.4,130039.6,143738.0,94313.4,12493.0,6940,39614.2,0
2921,2017-12-29 00:00:00+00:00,T11,188502.9,146124.7,165452.9,13330.9,5920.0,7934,8103,8036,...,-37053.6,-122976.5,20889.7,85117.9,122976.5,37053.6,20889.7,5539,43819.8,0
2922,2017-12-30 00:00:00+00:00,T11,65884.7,51194.5,57913.6,4789.3,4327.0,5416,5475,5453,...,-1886.9,-30372.6,6473.7,12643.7,30372.6,1886.9,6473.7,4220,22927.5,0


# 2. Clean Metacast data

Combine metamast data for both 2016 and 2017. Aggregate the metamast data into daily time slots

In [14]:
metmast = pd.concat([metmast_2016, metmast_2017], axis=0)
metmast['Timestamp'] = pd.to_datetime(metmast['Timestamp'])
metmast=metmast.set_index('Timestamp')

Drop the columns with low variance

In [15]:
cols_to_drop = get_signals_with_low_variance(metmast)  
metmast = metmast.drop(cols_to_drop, axis=1)
metmast= metmast.drop(["Min_Winddirection2", "Max_Winddirection2", "Avg_Winddirection2", "Var_Winddirection2"], axis=1)

In [16]:
agg_metmast=metmast.resample('D').sum()

In [17]:
agg_metmast['metamast_missing_values'] = agg_metmast.isnull().any(axis=1).astype(int)
metmast_clean=agg_metmast.fillna(method='bfill')
metmast_clean=metmast_clean.reset_index()


/var/folders/7b/1zsp53s91z97clkc70s5xcym0000gn/T/ipykernel_31723/3981064924.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  metmast_clean=agg_metmast.fillna(method='bfill')


In [18]:
metmast_clean

,Timestamp,Min_Windspeed1,Max_Windspeed1,Avg_Windspeed1,Var_Windspeed1,Min_Windspeed2,Max_Windspeed2,Avg_Windspeed2,Var_Windspeed2,Min_AmbientTemp,...,Max_Humidity,Avg_Humidity,Min_Precipitation,Max_Precipitation,Avg_Precipitation,Max_Raindetection,Anemometer1_Avg_Freq,Anemometer2_Avg_Freq,Pressure_Avg_Freq,metamast_missing_values
0,2016-01-01 00:00:00+00:00,617.5,1100.2,866.9,69.95,616.4,1112.7,873.2,73.17,2377.0,...,12698,12561.0,0,0,0,0,16889,17014,59950,0
1,2016-01-02 00:00:00+00:00,401.1,793.8,592.5,65.61,402.2,792.4,592.3,65.25,2144.0,...,12755,12635.0,22,85,54,0,11386,11376,59953,0
2,2016-01-03 00:00:00+00:00,523.3,1108.7,811.4,139.18,527.7,1109.1,813.9,138.68,2170.0,...,13671,13566.0,4,23,14,0,15764,15811,59603,0
3,2016-01-04 00:00:00+00:00,953.0,1936.8,1440.5,262.56,957.6,1940.1,1444.0,263.94,2407.0,...,12921,12749.0,6,30,17,0,28377,28462,58864,0
4,2016-01-05 00:00:00+00:00,1032.5,1864.1,1440.2,215.68,1020.8,1849.9,1429.9,215.81,1896.0,...,11252,11037.0,33,89,61,0,28373,28164,58591,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2017-12-27 00:00:00+00:00,1042.5,1902.9,1454.5,211.37,1049.8,1910.5,1462.0,214.27,2170.0,...,11590,11426.0,9,21,15,0,28645,28811,59293,0
727,2017-12-28 00:00:00+00:00,862.0,1478.7,1173.1,103.47,872.9,1491.0,1185.7,104.60,2224.0,...,12598,12483.0,2,10,6,0,23033,23268,59498,0
728,2017-12-29 00:00:00+00:00,576.6,912.3,746.0,36.36,570.1,909.7,742.0,36.86,2240.0,...,11954,11735.0,0,0,0,0,14460,14378,60189,0
729,2017-12-30 00:00:00+00:00,287.7,519.1,398.0,20.19,288.0,518.0,397.7,19.01,2214.0,...,11946,11756.0,0,4,2,0,7487,7472,60339,0


# Merge metamast data and signals data

In [19]:
def merge_signals_metmast(signals: pd.DataFrame, metmast: pd.DataFrame) -> pd.DataFrame:
    merged_df = pd.merge(signals.reset_index(drop=True), metmast.reset_index(drop=True), on="Timestamp", how="left")
    #merged_df.drop(columns=["index"], inplace=True)
    merged_df = merged_df[merged_df["missing_values"] == 0]
    merged_df = merged_df[merged_df["metamast_missing_values"] == 0]
    merged_df.drop(columns=["missing_values", "metamast_missing_values"], inplace=True)
    return merged_df


In [20]:
merged_df = merge_signals_metmast(signals_clean, metmast_clean)

In [21]:
merged_df.to_csv('./data/merged_signals_metmast.csv', index=False)

# 3. Preprocess Failure data

In [23]:
failures=pd.concat([failure_2016, failure_2017], axis=0)
failures['Timestamp'] = pd.to_datetime(failures['Timestamp'])
failures['Timestamp'] = failures['Timestamp'].dt.floor('d')


In [24]:
failures.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 0 to 11
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   Turbine_ID  28 non-null     object             
 1   Component   28 non-null     object             
 2   Timestamp   28 non-null     datetime64[ns, UTC]
 3   Remarks     28 non-null     object             
dtypes: datetime64[ns, UTC](1), object(3)
memory usage: 1.1+ KB


In [25]:
failures

,Turbine_ID,Component,Timestamp,Remarks
0,T01,GEARBOX,2016-07-18 00:00:00+00:00,Gearbox pump damaged
1,T06,GENERATOR,2016-07-11 00:00:00+00:00,Generator replaced
2,T06,GENERATOR,2016-07-24 00:00:00+00:00,Generator temperature sensor failure
3,T06,GENERATOR,2016-09-04 00:00:00+00:00,High temperature generator error
4,T06,GENERATOR,2016-10-27 00:00:00+00:00,Generator replaced
5,T06,GENERATOR,2016-10-02 00:00:00+00:00,Refrigeration system and temperature sensors i...
6,T06,HYDRAULIC_GROUP,2016-04-04 00:00:00+00:00,Error in pitch regulation
7,T07,GENERATOR_BEARING,2016-04-30 00:00:00+00:00,High temperature in generator bearing (replace...
8,T07,TRANSFORMER,2016-07-10 00:00:00+00:00,High temperature transformer
9,T07,TRANSFORMER,2016-08-23 00:00:00+00:00,High temperature transformer. Transformer refr...


In [86]:
days_lookback = 60
def create_failure_list(failures: pd.DataFrame, days_lookback: int, value_function, target_name: str = "Target") -> pd.DataFrame:
    failure_list = []

    for i in range(len(failures)):
        turbine_id = str(failures.iloc[i]["Turbine_ID"])
        failure_datetime = failures.iloc[i]["Timestamp"]
        components = failures.iloc[i]["Component"]
        rounded_datetime = failure_datetime.replace(hour=0, minute=0, second=0, microsecond=0)  # Round to the start of the day

        for j in range(days_lookback):
            delta = timedelta(days=j)
            new_datetime = rounded_datetime - delta
            datetime_formated = new_datetime.replace(tzinfo=timezone.utc)
            # Calculate the target value using the provided value_function
            target_value = value_function(j, days_lookback)
            failure_list.append([turbine_id, datetime_formated.isoformat(), components,target_value])
    
    failure_df = pd.DataFrame(failure_list, columns=["Turbine_ID", "Timestamp", "components",target_name])
    return failure_df

In [87]:
reg_target_name = "RUL (Target)"
class_target_name = "Failure (Target)"
regression_function = lambda i, j: i /j
classif_function = lambda i, j: 1
failure_df_reg = create_failure_list(failures, days_lookback, regression_function, reg_target_name)
failure_df_class = create_failure_list(failures, days_lookback, classif_function, class_target_name)


In [109]:
# RUL target column should show two decimal point
failure_df_reg[reg_target_name] = failure_df_reg[reg_target_name].apply(lambda x: round(x, 2))
failure_df_reg


,Turbine_ID,Timestamp,components,RUL (Target)
0,T01,2016-07-18 00:00:00+00:00,GEARBOX,0.00
1,T01,2016-07-17 00:00:00+00:00,GEARBOX,0.02
2,T01,2016-07-16 00:00:00+00:00,GEARBOX,0.03
3,T01,2016-07-15 00:00:00+00:00,GEARBOX,0.05
4,T01,2016-07-14 00:00:00+00:00,GEARBOX,0.07
...,...,...,...,...
1675,T11,2017-07-19 00:00:00+00:00,HYDRAULIC_GROUP,0.92
1676,T11,2017-07-18 00:00:00+00:00,HYDRAULIC_GROUP,0.93
1677,T11,2017-07-17 00:00:00+00:00,HYDRAULIC_GROUP,0.95
1678,T11,2017-07-16 00:00:00+00:00,HYDRAULIC_GROUP,0.97


In [110]:
failure_df_reg.to_csv('./data/failures_df.csv', index=False)

In [111]:
failure_df_reg['Timestamp'] = pd.to_datetime(failure_df_reg['Timestamp'])
failure_df_class['Timestamp'] = pd.to_datetime(failure_df_class['Timestamp'])

In [112]:
labeled_df_temp = pd.merge(merged_df.reset_index(drop=True), failure_df_reg.reset_index(drop=True), on=["Turbine_ID", "Timestamp"], how="left"); 
labeled_df = pd.merge(labeled_df_temp.reset_index(drop=True), failure_df_class.reset_index(drop=True), on=["Turbine_ID", "Timestamp"], how="left"); 

In [113]:
labeled_df[reg_target_name].fillna(1.0, inplace = True)
labeled_df[class_target_name] = labeled_df[class_target_name].fillna(0).astype(int)

/var/folders/7b/1zsp53s91z97clkc70s5xcym0000gn/T/ipykernel_31723/4169923202.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  labeled_df[reg_target_name].fillna(1.0, inplace = True)


In [117]:
labeled_df.drop_duplicates(inplace=True)
labeled_df.drop(columns=[ "components_y"], inplace=True)

In [118]:
#rename component_x as component
labeled_df.rename(columns = {'components_x':'components'}, inplace = True)


In [119]:
labeled_df.to_csv('./data/labeled_data.csv', index=False)

In [120]:
plt.rcParams["figure.figsize"] = (10,5)